In [5]:
import pandas as pd
import  duckdb

from duckdb_flow.etl import preprocess_data, get_processed_files
from duckdb_flow.create_database import create_database

%load_ext autoreload

The autoreload module is not an IPython extension.


In [2]:
STAGING_PATH = "./data"
DATABASE_PATH = "./database/DDBB_duckdb.duckdb"

In [3]:
create_database()

'Database Created'

In [4]:
processed_files = get_processed_files()
movies_df, directing_data, writing_data  = preprocess_data(processed_files)

Created  directing_df  dataframe.
Appending:  963  rows...
Appending:  993  rows...
Appending:  951  rows...
Appending:  1049  rows...
Appending:  974  rows...
Appending:  1001  rows...
Appending:  1035  rows...
Appending:  993  rows...
Created  writing_df  dataframe.


In [10]:
STAGING_PATH = "./data"
DATABASE_PATH = "./database/DDBB_duckdb.duckdb"


con = duckdb.connect(database=DATABASE_PATH, read_only=False)
result = con.execute('''select * from movies''').fetch_df()
result.shape


(10000, 8)